# 使用future处理并发

## 17.1 示例：网络下载的风格

### 17.1.1 依序下载的脚本

In [12]:
import os
import time
import sys

import requests

POP20_CC = ('CN IN US ID BR PK NG BD RU JP MX PH VN ET EG DE IR TR CD FR').split() 

BASE_URL = 'http://flupy.org/data/flags'

DEST_DIR = './downloads/'

def save_flag(img, filename):
    path = os.path.join(DEST_DIR, filename)
    with open(path, 'wb') as fp:
        fp.write(img)


def get_flag(cc):
    url = '{}/{cc}{cc}.gif'.format(BASE_URL, cc=cc.lower())
    resp = requests.get(url)
    return resp.content


def show(text):
    print(text, end=' ')
    sys.stdout.flush()


def download_many(cc_list):
    for cc in sorted(cc_list):
        image = get_flag(cc)
        show(cc)
        save_flag(image, cc.lower() + '.gif')

    return len(cc_list)


def main(download_many):
    t0 = time.time()
    count = download_many(POP20_CC)
    elapsed = time.time() - t0
    msg = '\n{} flags download in {:.2f}s'
    print(msg.format(count, elapsed))


if __name__ == '__main__':
    main(download_many)

BD BR CD CN DE EG ET FR ID IN IR JP MX NG PH PK RU TR US VN 
20 flags download in 36.51s


In [13]:
print('hello', end=' ')
sys.stdout.flush()
print('world', end=' ')

hello world 

### 17.1.2 使用concurrent.futures模块下载

In [14]:
from concurrent import futures

MAX_WORKERS = 20


def download_one(cc):
    image = get_flag(cc)
    show(cc)
    save_flag(image, cc.lower() + '.gif')
    return cc


def download_many(cc_list):
    workers = min(MAX_WORKERS, len(cc_list))
    with futures.ThreadPoolExecutor(workers) as executor:
        res = executor.map(download_one, sorted(cc_list))

    return len(list(res))


if __name__ == '__main__':
    main(download_many)

CD IR PK TR BD US RU PH DE IN VN NG MX ID CN JP EG ET BR FR 
20 flags download in 1.80s


### 17.1.3 future在哪里

In [15]:
def download_many(cc_list):
    cc_list = cc_list[:5]
    with futures.ThreadPoolExecutor(max_workers=3) as executor:
        to_do = []
        for cc in sorted(cc_list):
            future = executor.submit(download_one, cc)
            to_do.append(future)
            msg = 'Scheduled for {}: {}'
            print(msg.format(cc, future))

        results = []
        for future in futures.as_completed(to_do):
            res = future.result()
            msg = '{} result: {!r}'
            print(msg.format(future, res))
            results.append(res)

    return len(results)


if __name__ == '__main__':
    main(download_many)

Scheduled for BR: <Future at 0x187ae6c8910 state=running>
Scheduled for CN: <Future at 0x187af5568f0 state=running>
Scheduled for ID: <Future at 0x187af554280 state=running>
Scheduled for IN: <Future at 0x187af556c20 state=pending>
Scheduled for US: <Future at 0x187af555f90 state=pending>
BR ID <Future at 0x187ae6c8910 state=finished returned str> result: 'BR'
<Future at 0x187af554280 state=finished returned str> result: 'ID'
CN <Future at 0x187af5568f0 state=finished returned str> result: 'CN'
US <Future at 0x187af555f90 state=finished returned str> result: 'US'
IN <Future at 0x187af556c20 state=finished returned str> result: 'IN'

5 flags download in 3.34s


## 17.3 使用concurrent.futures模块启动进程

[A-7](./arcfour_futures.py)
<br>
[A-9](./sha_futures.py)

## 17.4 实验Executor.map方法

In [4]:
from time import sleep, strftime
from concurrent import futures


def display(*args):
    print(strftime('[%H:%M:%S]'), end=' ')
    print(*args)


def loiter(n):
    msg = '{}loiter({}): doing nothing for {}s...'
    display(msg.format('\t' * n, n, n))
    sleep(n)
    msg = '{}loiter({}): done.'
    display(msg.format('\t' * n, n))
    return n * 10 


def main():
    display('Script starting.')
    executor = futures.ThreadPoolExecutor(max_workers=3)
    results = executor.map(loiter, range(5))
    display('results:', results)
    display('Waiting for individual results:')
    for i, result in enumerate(results):
        display('result {}: {}'.format(i, result))
    

main()

[16:21:52] Script starting.
[16:21:52] loiter(0): doing nothing for 0s...
[16:21:52] loiter(0): done.
[16:21:52] 	loiter(1): doing nothing for 1s...
[16:21:52] 		loiter(2): doing nothing for 2s...
[16:21:52] 			loiter(3): doing nothing for 3s...
[16:21:52] results: <generator object Executor.map.<locals>.result_iterator at 0x000002764FD62A40>
[16:21:52] Waiting for individual results:
[16:21:52] result 0: 0
[16:21:53] 	loiter(1): done.
[16:21:53] 				loiter(4): doing nothing for 4s...
[16:21:53] result 1: 10
[16:21:54] 		loiter(2): done.
[16:21:54] result 2: 20
[16:21:55] 			loiter(3): done.
[16:21:55] result 3: 30
[16:21:57] 				loiter(4): done.
[16:21:57] result 4: 40


## 17.5 显示下载进度并处理错误

[flags2_common.py](./flags2_common.py)
<br>
[flags2_sequential.py](./flags2_sequential.py)
<br>
[flags2_threadpool.py](./flags2_threadpool.py)

In [7]:
import time
from tqdm import *

for i in tqdm(range(1000)):
    time.sleep(0.01)

100%|██████████| 1000/1000 [00:15<00:00, 64.17it/s]
